Note: Responses from local models can be quite slow, especially with 8-bit quantization.

With 4bit quantization, `HuggingFaceH4/zephyr-7b-alpha` uses about 8GB of VRAM and spiked to 14GB of RAM when loading the model, then settled around 5GB. I used a T4 instance for this notebook.

In [ ]:
!pip install llama-index llama-index-llms-huggingface llama-index-embeddings-huggingface transformers accelerate bitsandbytes llama-index-readers-web

## Setup

### Data

In [1]:
from llama_index.readers.web import BeautifulSoupWebReader

url = "https://www.theverge.com/2023/9/29/23895675/ai-bot-social-network-openai-meta-chatbots"

documents = BeautifulSoupWebReader().load_data([url])

/Users/dpadmana/envs/llamaindex-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### LLM

This should run on a T4 instance on the free tier

In [3]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)


def messages_to_prompt(messages):
  prompt = ""
  for message in messages:
    if message.role == 'system':
      prompt += f"<|system|>\n{message.content}</s>\n"
    elif message.role == 'user':
      prompt += f"<|user|>\n{message.content}</s>\n"
    elif message.role == 'assistant':
      prompt += f"<|assistant|>\n{message.content}</s>\n"

  # ensure we start with a system prompt, insert blank if needed
  if not prompt.startswith("<|system|>\n"):
    prompt = "<|system|>\n</s>\n" + prompt

  # add final assistant prompt
  prompt = prompt + "<|assistant|>\n"

  return prompt


llm = HuggingFaceLLM(
    model_name="HuggingFaceH4/zephyr-7b-alpha",
    tokenizer_name="HuggingFaceH4/zephyr-7b-alpha",
    query_wrapper_prompt=PromptTemplate("<|system|>\n</s>\n<|user|>\n{query_str}</s>\n<|assistant|>\n"),
    context_window=3900,
    max_new_tokens=256,
    # model_kwargs={"quantization_config": quantization_config},
    # tokenizer_kwargs={},
    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    messages_to_prompt=messages_to_prompt,
    device_map="auto",
)

model.safetensors.index.json: 100%|██████████| 23.9k/23.9k [00:00<00:00, 6.77MB/s]
model-00001-of-00008.safetensors: 100%|██████████| 1.89G/1.89G [03:33<00:00, 8.85MB/s]
model-00002-of-00008.safetensors: 100%|██████████| 1.95G/1.95G [03:42<00:00, 8.75MB/s]
model-00003-of-00008.safetensors: 100%|██████████| 1.98G/1.98G [04:07<00:00, 8.01MB/s]
model-00004-of-00008.safetensors: 100%|██████████| 1.95G/1.95G [03:07<00:00, 10.4MB/s]
model-00005-of-00008.safetensors: 100%|██████████| 1.98G/1.98G [03:29<00:00, 9.45MB/s]
model-00006-of-00008.safetensors: 100%|██████████| 1.95G/1.95G [03:20<00:00, 9.71MB/s]
model-00007-of-00008.safetensors: 100%|██████████| 1.98G/1.98G [03:18<00:00, 9.97MB/s]
model-00008-of-00008.safetensors: 100%|██████████| 816M/816M [01:15<00:00, 10.8MB/s]
/Users/dpadmana/envs/llamaindex-env/lib/python3.9/site-packages/accelerate/utils/modeling.py:1341: UserWarning: Current model requires 1006640640 bytes of buffer for offloaded layers, which seems does not fit any GPU's rema

In [4]:
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.llm = llm
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

config.json: 100%|██████████| 743/743 [00:00<00:00, 214kB/s]
model.safetensors: 100%|██████████| 133M/133M [00:14<00:00, 9.51MB/s] 
tokenizer_config.json: 100%|██████████| 366/366 [00:00<00:00, 174kB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 528kB/s]
tokenizer.json: 100%|██████████| 711k/711k [00:00<00:00, 2.66MB/s]
special_tokens_map.json: 100%|██████████| 125/125 [00:00<00:00, 53.9kB/s]


### Index Setup

In [5]:
documents

[Document(id_='https://www.theverge.com/2023/9/29/23895675/ai-bot-social-network-openai-meta-chatbots', embedding=None, metadata={'URL': 'https://www.theverge.com/2023/9/29/23895675/ai-bot-social-network-openai-meta-chatbots'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="The synthetic social network is coming - The VergeSkip to main contentThe VergeThe Verge logo.The Verge homepageThe Verge homepageThe VergeThe Verge logo./Tech/Reviews/Science/Entertainment/MoreMenuExpandThe VergeThe Verge logo.MenuExpandPlatformer/Artificial Intelligence/TechThe synthetic social network is comingThe synthetic social network is coming / Between ChatGPT’s surprisingly human voice and Meta’s AI characters, our feeds may be about to change foreverBy  Casey Newton, a contributing editor who has been writing about tech for over 10 years. He founded Platformer, a newsletter about Big Tech and democracy. Sep 29, 2023, 1:30 PM UTCShare this story Image: Álvaro Bernis

In [6]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents(documents)

In [7]:
from llama_index.core import SummaryIndex

summary_index = SummaryIndex.from_documents(documents)

### Helpful Imports / Logging

In [9]:
from llama_index.core.response.notebook_utils import display_response

In [10]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Basic Query Engine

### Compact (default)

In [11]:
query_engine = vector_index.as_query_engine(response_mode="compact")

response = query_engine.query("How do OpenAI and Meta differ on AI tools?")

display_response(response)

/Users/dpadmana/envs/llamaindex-env/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/Users/dpadmana/envs/llamaindex-env/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** OpenAI tends to present its products as productivity tools, while Meta is in the entertainment business. However, both companies are building LLMs (large language models) and have recently revealed their own uses for generative AI and voices. While OpenAI's focus is on simple utilities for getting things done, Meta has unveiled 28 personality-driven chatbots to be used in its messaging apps, featuring celebrities such as Charli D'Amelio, Dwyane Wade, Kendall Jenner, MrBeast, Snoop Dogg, Tom Brady, and Paris Hilton.

### Refine

In [12]:
query_engine = vector_index.as_query_engine(response_mode="refine")

response = query_engine.query("How do OpenAI and Meta differ on AI tools?")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

### Tree Summarize

In [ ]:
query_engine = vector_index.as_query_engine(response_mode="tree_summarize")

response = query_engine.query("How do OpenAI and Meta differ on AI tools?")

display_response(response)

**`Final Response:`** OpenAI and Meta both have their own uses for generative AI and voices, but they differ in their approach to AI tools. OpenAI tends to present its products as productivity tools, while Meta is in the entertainment business and has found its own uses for LLMs. OpenAI's latest updates for ChatGPT include a voice feature that allows users to interact with the large language model via voice, while Meta has unveiled 28 personality-driven chatbots to be used in its messaging apps. However, the article suggests that the characters in Meta's chatbots may not have the same level of novelty value as the synthetic versions of celebrities, and it remains to be seen how popular these chatbots will be. Overall, both companies are exploring the potential of AI in social networking and feeds, but the article raises questions about whether this technology will feel personalized, engaging, and entertaining or uncanny, hollow, and junky.

## Router Query Engine

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    vector_index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts."
    )
)

summary_tool = QueryEngineTool(
    summary_index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document."
    )
)

### Single Selector

In [ ]:
from llama_index.core.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    select_multi=False
)

response = query_engine.query("What was mentioned about Meta?")

display_response(response)

**`Final Response:`** Meta is building LLMs and personality-driven chatbots to be used in their messaging apps. They have unveiled 28 personality-driven chatbots, and celebrities including Charli D’Amelio, Dwyane Wade, Kendall Jenner, MrBeast, Snoop Dogg, Tom Brady, and Paris Hilton lent their voices to their effort. The company is in the entertainment business and is using generative AI and voices for their chatbots. The article suggests that this technology is new enough that celebrities aren't yet entrusting their entire personas to Meta for safekeeping, but they are giving people a taste of what it's like to talk to AI versions of celebrities to iron out any kinks before delivering the man himself. The potential seems very real, and the article raises questions about how many hours fans would spend talking to a digital version of Taylor Swift this year, if they could.

### Multi Selector

In [ ]:
from llama_index.core.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    select_multi=True,
)

response = query_engine.query("What was mentioned about Meta? Summarize with any other companies mentioned in the entire document.")

display_response(response)

**`Final Response:`** Meta is building LLMs and personality-driven chatbots to be used in their messaging apps. They have unveiled 28 personality-driven chatbots, including characters voiced by celebrities such as Charli D’Amelio, Dwyane Wade, Kendall Jenner, MrBeast, Snoop Dogg, Tom Brady, and Paris Hilton. The characters will be placed on every major surface of its products, including Facebook pages and Instagram accounts. Other companies mentioned in the document include OpenAI, Google, Alexa, and the Google assistant.

## SubQuestion Query Engine

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    vector_index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts."
    )
)

summary_tool = QueryEngineTool(
    summary_index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document."
    )
)

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from llama_index.core.query_engine import SubQuestionQueryEngine

query_engine = SubQuestionQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    verbose=True,
)

response = query_engine.query("What was mentioned about Meta? How Does it differ from how OpenAI is talked about?")

display_response(response)

Generated 5 sub questions.
[vector_search] Q: What was mentioned about Meta in a recent news article?
[vector_search] A: In a recent news article, it was mentioned that Meta is building LLMs (large language models) and has unveiled 28 personality-driven chatbots to be used in their messaging apps. These chatbots are powered by generative AI and voices, and celebrities including Charli D’Amelio, Dwyane Wade, Kendall Jenner, MrBeast, Snoop Dogg, Tom Brady, and Paris Hilton lent their voices to their effort. The article suggests that this technology is new enough that celebrities aren't yet entrusting their entire personas to Meta for safekeeping, but they are giving people a taste of what it's like to talk to AI versions of themselves. The potential for these chatbots is significant, and the article raises questions about how many hours people would spend talking to a digital version of Taylor Swift this year, if they could. The article also suggests that this technology could lead to a 

**`Final Response:`** In recent news articles, it was mentioned that Meta is building LLMs (large language models) and has unveiled 28 personality-driven chatbots to be used in their messaging apps. These chatbots are powered by generative AI and voices, and celebrities including Charli D’Amelio, Dwyane Wade, Kendall Jenner, MrBeast, Snoop Dogg, Tom Brady, and Paris Hilton lent their voices to their effort. In contrast, OpenAI's latest updates for ChatGPT include a voice feature and the ability to upload images and ask questions about them, but OpenAI tends to present its products as productivity tools, while Meta is in the entertainment business. While both companies are building LLMs, Meta's focus is on entertainment and creating AI characters for social networking, while OpenAI's focus is on productivity and AI-generated imagery.

## SQL Query Engine

Here, we download and use a sample SQLite database with 11 tables, with various info about music, playlists, and customers. We will limit to a select few tables for this test.

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!curl https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip -O /content/chinook.zip
!unzip /content/chinook.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  298k  100  298k    0     0  2409k      0 --:--:-- --:--:-- --:--:-- 2426k
curl: (3) URL using bad/illegal format or missing URL
Archive:  /content/chinook.zip
  inflating: chinook.db              


In [ ]:
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, select, column

engine = create_engine("sqlite:////content/chinook.db")

In [ ]:
from llama_index.core import SQLDatabase

sql_database = SQLDatabase(engine)

In [ ]:
from llama_index.core.indices.struct_store import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["albums", "tracks", "artists"],
)

In [ ]:
response = query_engine.query("What are some albums? Limit to 5.")

display_response(response)

**`Final Response:`** Based on the query results, some albums are:
1. Koyaanisqatsi (Soundtrack from the Motion Picture) by Philip Glass Ensemble (AlbumId: 347)
2. Mozart: Chamber Music by Nash Ensemble (AlbumId: 346)
3. Monteverdi: L'Orfeo by C. Monteverdi, Nigel Rogers - Chiaroscuro; London Baroque; London Cornett & Sackbu (AlbumId: 345)
4. Schubert: The Late String Quartets & String Quintet (3 CD's) by Emerson String Quartet (AlbumId: 344)
5. Respighi:Pines of Rome by Eugene Ormandy (AlbumId: 343)

In [ ]:
response = query_engine.query("What are some artists? Limit it to 5.")

display_response(response)

**`Final Response:`** Some artists, limited to 5, are: A Cor Do Som, AC/DC, Aaron Copland & London Symphony Orchestra, Aaron Goldberg, and Academy of St. Martin in the Fields & Sir Neville Marriner.

This last query should be a more complex join

In [ ]:
response = query_engine.query("What are some tracks from the artist AC/DC? Limit it to 3")

display_response(response)

**`Final Response:`** Some tracks from the artist AC/DC that we'll be discussing today are "Bad Boy Boogie," "Breaking The Rules," and "C.O.D." These songs are from different albums, but they all showcase the iconic sound of AC/DC.

In [ ]:
print(response.metadata['sql_query'])

SELECT tracks.Name
FROM tracks
INNER JOIN albums ON tracks.AlbumId = albums.AlbumId
INNER JOIN artists ON albums.ArtistId = artists.ArtistId
WHERE artists.Name = 'AC/DC'
GROUP BY tracks.Name
ORDER BY tracks.Name ASC
LIMIT 3;


## Programs

Depending the LLM, you will have to test with either `OpenAIPydanticProgram` or `LLMTextCompletionProgram`

In [ ]:
from typing import List
from pydantic import BaseModel

from llama_index.core.program import LLMTextCompletionProgram

class Song(BaseModel):
    """Data model for a song."""

    title: str
    length_seconds: int


class Album(BaseModel):
    """Data model for an album."""

    name: str
    artist: str
    songs: List[Song]

In [ ]:
from llama_index.core.output_parsers import PydanticOutputParser

prompt_template_str = """\
Generate an example album, with an artist and a list of songs. \
Using the movie {movie_name} as inspiration.\
"""
program = LLMTextCompletionProgram.from_defaults(
    output_parser=PydanticOutputParser(Album),
    prompt_template_str=prompt_template_str,
    llm=llm,
    verbose=True,
)

In [ ]:
output = program(movie_name="The Shining")

In [ ]:
print(output)

name='The Shining Soundtrack' artist='Wendy Carlos' songs=[Song(title='Main Title', length_seconds=2), Song(title='The Shining', length_seconds=10), Song(title='The Maze', length_seconds=12), Song(title='The Redrum', length_seconds=10), Song(title='The Maze (Reprise)', length_seconds=6), Song(title='The Shining (End Title)', length_seconds=10)]


## Data Agent

Similar to programs, OpenAI LLMs will use `OpenAIAgent`, while other LLMs will use `ReActAgent`.

In [ ]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(
    [vector_tool, summary_tool],
    llm=llm,
    verbose=True
)

Some inputs are hallucinated, causing issues with responses. Likely a better system prompt or tool descriptions could help.

In [ ]:
response = agent.chat("Hello!")
print(response)

Thought: I am designed to help with a variety of tasks.
Action: vector_search
Action Input: {'text': 'Hello!', 'num_beams': 5}
Observation: This query is not related to the given context information. The query provided is a hypothetical example of a query that could be made to a language model, and does not have any relevance to the article discussed.
Response: The input provided is not related to the given context information.
The input provided is not related to the given context information.


In [ ]:
response = agent.chat("What was mentioned about Meta? How Does it differ from how OpenAI is talked about?")
print(response)

Thought: I need to use a tool to help me answer the question.
Action: vector_search
Action Input: {'text': 'Meta and OpenAI', 'num_beams': 5}
Observation: In the given context, the query "Meta and OpenAI" refers to two companies that are building LLMs (language learning models) and using generative AI and voices. Meta is in the entertainment business and has unveiled 28 personality-driven chatbots to be used in its messaging apps, while OpenAI announced updates for ChatGPT, including the ability to interact with it via voice and upload images for questions. Both companies are exploring the potential of synthetic companions that can offer coaching, tutoring, therapy, and entertainment, and the rise of a new era in the consumer internet.
Response: In terms of how they differ, Meta's focus is on using generative AI and voices to create personality-driven chatbots for its messaging apps, while OpenAI's focus is on improving its ChatGPT model and exploring the potential of synthetic compani